In [27]:
import pandas as pd
from google.colab import files

# Step 1: Upload your CSV file
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Step 2: Read and parse the data
df = pd.read_csv(filename)
df['datetimeUtc'] = pd.to_datetime(df['datetimeUtc'], utc=True)

# Step 3: Add a 'date' column for easier filtering (keep date only, ignore time for grouping)
df['date'] = df['datetimeUtc'].dt.date

# Step 4: Select the first timestamp for each day (this is the accurate timestamp for that day)
df_first_values = df.groupby(['date', 'location_id', 'location_name', 'latitude', 'longitude'], as_index=False).first()

# Step 5: Pivot the table to get each parameter as a separate column
df_pivot = df.pivot_table(
    index=['location_id', 'location_name', 'date', 'latitude', 'longitude'],
    columns='parameter',
    values='value',
    aggfunc='first'  # Only one value per day per parameter
).reset_index()

# Step 6: Rename columns to the required format
df_pivot = df_pivot.rename(columns={
    'pm25': 'PM2.5',
    'pm10': 'PM10',
    'no2': 'NO₂',
    'co': 'CO',
    'o3': 'O₃',
    'no': 'NO'
})

# Step 7: Fetch the correct timestamp for each day
df_first_values['Timestamp'] = df_first_values['datetimeUtc'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')

# Step 8: Merge the first timestamp values back to the pivoted data
final_df = pd.merge(df_pivot, df_first_values[['date', 'location_id', 'location_name', 'Timestamp']],
                     on=['date', 'location_id', 'location_name'], how='left')

# Step 9: Reorder columns as per the desired output
final_df = final_df[[
    'location_id', 'location_name', 'Timestamp', 'PM2.5', 'PM10', 'NO₂', 'CO', 'O₃', 'NO', 'latitude', 'longitude'
]]

# Step 10: Save the result to a CSV file and prepare for download
output_filename = 'daily_air_quality_data_with_correct_timestamp.csv'
final_df.to_csv(output_filename, index=False)

# Step 11: Download the CSV file
files.download(output_filename)


Saving openaq_location_6945_measurments.csv to openaq_location_6945_measurments (11).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>